In [ ]:
import os
import pandas as pd
import pickle as pkl
import sys
from matplotlib import pyplot as plt
import numpy as np
import cv2

##Only include while running on colab
#from google.colab import drive
#drive.mount("/gdrive", force_remount=True)

#Directory path to save pickle files.
outFile='/gdrive/My Drive/URDU_V2_Exp/wap_exp/data_inv/train_images_inv.pkl'
outFile2='/gdrive/My Drive/URDU_V2_Exp/wap_exp/data_inv/train_labels_inv.pkl'
oupFp_feature=open(outFile,'wb')
oupFp_feature2=open(outFile2,'wb')

#This noise function is used to add noise in images.
def add_salt_pepper_noise(X_imgs):
    # Need to produce a copy as to not modify the original image
    X_imgs_copy = X_imgs.copy()
    
    row, col= X_imgs_copy.shape
    salt_vs_pepper = 0.2
    amount = 0.04
    num_salt = np.ceil(amount * X_imgs_copy.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * X_imgs_copy.size * (1.0 - salt_vs_pepper))
    
   
    # Add Salt noise
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in X_imgs.shape]
    X_imgs[coords[0], coords[1]] = 1

    # Add Pepper noise
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in X_imgs.shape]
    X_imgs[coords[0], coords[1]] = 0
    return X_imgs
  
#Automatically generate dictionary array by reading training labels.
def load_dict(labelfile):
    count=0
    df=pd.read_excel(labelfile)
    lexicon={}
    lex_ind=[]
    j=1
    key=1
    for i in df.index: 
      caption=df['Revised'][i]
      count=count+1
      slen=len(caption)
      i=0
      while(i<slen):
        ss=caption[i]
        
        if(ss=='ة' or ss=='ۃ'):
          ss='ت'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ہ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1
        if(ss=='ي'):
          ss='ے'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ی'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1
        if(ss=='ؤ' or ss=='ﺅ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
           
            j=j+1
            key=key+1

          ss='و'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

        if(ss=='ئ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ی'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

        if(ss=='ۂ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ہ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

        if(ss=='ۓ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ے'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1
        if(ss=='أ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ا'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

        if(ss=='ﷲ'):
          ss='ا'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

          ss='ل'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1 
          ss='ل'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1 
          ss='ہ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1 

        if(ss=='ٔ' or ss=='ٔ'):
          ss='ٔ'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1

        if(ss=='\u200f' or ss=='\u200c' or ss=='\ufeff' or ss==' ' or ss=='\n'):
          ss=' '  
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1
             
        if(ss=='-' or ss=='۔'):
          ss='۔'
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1  
        else:
          if ss not in lexicon:
            lexicon[ss]=int(key)
            lex_ind.append(j)
            j=j+1
            key=key+1
       
        i=i+1
  
    return lexicon,lex_ind

#This function reads labels and corresponding images from given paths of data. Urdu Llabels are converted to numbers using dictionary. 
#Images are resized and some noise is also added which helps to reduce overfitting during the training process.
def  load_labels_from_folder(imgfolder,labelfile,dictionary):  
  
    labels=[]
    images={}
    count=0
    scpFile=open(labelfile)
    df=pd.read_excel(labelfile)
    split_ch='-'
    
    i=0
    for i in df.index:
      line=df['Num'][i]#Num
      caption=df['Revised'][i]# Caption
      slen=len(caption)
      j=0

      image_file = imgfolder + line.strip() + '.png'
      img = cv2.imread(image_file,-1)
      
      if img is None:
        print(image_file)
      else:
        if len(img.shape)>2:
          img= cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2GRAY)
        height=img.shape[0]
        width=img.shape[1]
        #print(width)
        if(width<300):
          result = np.ones([img.shape[0], img.shape[1]*2])*255
          result[0:img.shape[0],img.shape[1]:img.shape[1]*2]=img
          img=cv2.resize(result, dsize=(800,100), interpolation = cv2.INTER_AREA)      
        else:
          img=cv2.resize(img, dsize=(800,100), interpolation = cv2.INTER_AREA)
          
        img=add_salt_pepper_noise(img)
        images[count]=img
        count=count+1
        

        w_list=[]
      
        w=0
        while(w < slen):
          ss=caption[w]
          if(ss=='ي'):
            ss='ے'
            if ss in dictionary:
              w_list.append(dictionary[ss])
            ss='ی'
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss=='ة'):
            ss='ت'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ہ'
            if ss in dictionary:
              w_list.append(dictionary[ss])
          elif(ss=='ؤ' or ss=='ﺅ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])
            ss='و'
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss=='ئ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ی'
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss=='ۂ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ہ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss=='ۓ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ے'
            if ss in dictionary:
              w_list.append(dictionary[ss])
          elif(ss=='أ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ا'
            if ss in dictionary:
              w_list.append(dictionary[ss])
          elif(ss=='ﷲ'):
            ss='ا'
            if ss in dictionary:
              w_list.append(dictionary[ss])

            ss='ل'
            if ss in dictionary:
              w_list.append(dictionary[ss]) 
            ss='ل'
            if ss in dictionary:
              w_list.append(dictionary[ss]) 
            ss='ہ'
            if ss in dictionary:
              w_list.append(dictionary[ss]) 

          elif(ss=='ٔ' or ss=='ٔ'):
            ss='ٔ'
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss==' '):
            ss=' '  
            if ss in dictionary:
              w_list.append(dictionary[ss])

          elif(ss=='-' or ss=='۔'):
            ss='۔'
            if ss in dictionary:
              w_list.append(dictionary[ss])
          else:
            if ss in dictionary:
              w_list.append(dictionary[ss])
          w=w+1

        
        xx=w_list[::-1]
        xx.append(0)
        labels.append(xx)
        i=i+1

      if (np.mod(count, 200) == 0):
        print(count)  
    return images, labels  

      
worddicts,wordindexes = load_dict('/gdrive/My Drive/URDU_V2_Exp/V2_data/train_labels_combined.xlsx') 
print(worddicts)#dictionary
images,labels=load_labels_from_folder('/gdrive/My Drive/URDU FINAL/variable images experiment/train_lines/','/gdrive/My Drive/URDU_V2_Exp/V2_data/train_labels_combined.xlsx',worddicts)


pkl.dump(images,oupFp_feature)
pkl.dump(labels,oupFp_feature2)
print ('save file done')
oupFp_feature.close()
oupFp_feature2.close()